In [1]:
import numpy as np

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import kernels

In [2]:
import solution
from solution import Model
from solution import cv_eval


In [3]:
#copy code to override default classes

In [4]:
#def main():
train_x_name = "train_x.csv"
train_y_name = "train_y.csv"

train_x = np.loadtxt(train_x_name, delimiter=',')
train_y = np.loadtxt(train_y_name, delimiter=',')

# load the test dateset
test_x_name = "test_x.csv"
test_x = np.loadtxt(test_x_name, delimiter=',')

In [ ]:
### TODO add a simple outer train, test split to select the hyperparameters (kernel type etc) in the inner CV loop 
kernel = kernels.sklearn_best()
K_cv = 10
cv_m = cv_eval(cv_splits = K_cv, kernel = kernel, use_skit_learn = True) # Choose whether to use custom model or scikit learn
val_cost_array = cv_m.run_cross_validation(train_x, train_y)

print(np.mean(val_cost_array))
print(np.std(val_cost_array))

[[-1.         -1.          0.75400097]
 [-1.         -0.98657718  0.83958557]
 [-1.         -0.97315436  0.81706415]
 ...
 [-0.47651007 -0.73154362  0.22523587]
 [ 0.5033557   0.53020134  0.49737222]
 [ 0.00671141  0.5033557   0.29129925]]
(17250, 3)
(17250, 3)
i=0
training cost fn   0.025794
val cost fn        0.029787


i=1
